In [1]:
import pandas as pd
df = pd.read_csv('ar41_for_ulb_cleaned.csv')
df['timestamps_UTC'] = pd.to_datetime(df['timestamps_UTC'])
df['date_hour'] = df['timestamps_UTC'].dt.to_period('H')
agg_df = pd.read_csv('ar41_for_ulb_round.csv')
agg_df['date_hour'] = pd.to_datetime(agg_df['date_hour'])
agg_df['date_hour'] = agg_df['date_hour'].dt.to_period('H')

merged_df = pd.merge(df, agg_df, on=['mapped_veh_id', 'date_hour'], how='left')
merged_df.to_csv('grouped.csv', index=False)

In [5]:
df = pd.read_csv('grouped.csv')
columns_to_drop = ['date_hour', 'lat_y', 'lon_y']
df.drop(columns_to_drop, axis=1, inplace=True)
df

,mapped_veh_id,timestamps_UTC,lat,lon,RS_E_InAirTemp_PC1,RS_E_InAirTemp_PC2,RS_E_OilPress_PC1,RS_E_OilPress_PC2,RS_E_RPM_PC1,RS_E_RPM_PC2,...,main_temp_min,main_temp_max,wind_speed,wind_deg,wind_gust,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,181.0,2023-08-01 03:44:12,50.769818,3.872114,27.0,23.0,255.0,238.0,794.0,801.0,...,287.64,288.74,5.89,255.0,12.99,97.0,804.0,Clouds,overcast clouds,04n
1,143.0,2023-08-01 06:36:29,51.039993,3.693429,33.0,32.0,272.0,324.0,802.0,804.0,...,288.09,289.32,5.27,251.0,10.46,56.0,803.0,Clouds,broken clouds,04d
2,183.0,2023-08-24 06:53:54,50.742203,3.602035,31.0,33.0,234.0,182.0,799.0,802.0,...,290.32,292.38,1.34,80.0,2.68,95.0,804.0,Clouds,overcast clouds,04d
3,177.0,2023-08-01 13:53:38,50.930914,5.327132,35.0,38.0,220.0,244.0,794.0,801.0,...,292.08,294.83,6.17,270.0,NaN,75.0,803.0,Clouds,broken clouds,04d
4,143.0,2023-08-24 07:02:30,51.180773,3.575259,41.0,34.0,227.0,282.0,806.0,800.0,...,291.57,294.29,2.04,125.0,3.65,100.0,804.0,Clouds,overcast clouds,04d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17666542,180.0,2023-05-28 19:28:59,50.919864,3.791125,40.0,36.0,238.0,238.0,800.0,797.0,...,288.59,293.73,6.02,33.0,10.80,51.0,803.0,Clouds,broken clouds,04d
17666543,180.0,2023-05-28 19:54:41,51.035243,3.710695,41.0,31.0,244.0,241.0,790.0,801.0,...,288.59,293.73,6.02,33.0,10.80,51.0,803.0,Clouds,broken clouds,04d
17666544,122.0,2023-05-28 20:23:20,51.015686,3.775316,52.0,42.0,3.0,3.0,0.0,0.0,...,287.09,291.20,6.20,33.0,11.94,39.0,802.0,Clouds,scattered clouds,03n
17666545,177.0,2023-05-28 20:46:00,51.189586,5.109380,27.0,38.0,417.0,469.0,1441.0,1454.0,...,288.99,292.29,6.21,33.0,12.24,19.0,801.0,Clouds,few clouds,02n


In [6]:
df.to_csv('grouped.csv', index=False)

In [2]:
import pandas as pd

df = pd.read_csv('grouped.csv')[['lat', 'lon']]

# Function to assign coordinates to 500-meter cells
def assign_to_cells(latitude, longitude):
    # Define the size of each cell (500 meters)
    cell_size = 0.000898311
    # cell_size = 0.001796622  # 200 meters
    # cell_size = 0.004491556  # Adjust this value based on your needs
    
    # Assign each coordinate to a cell
    cell_latitude = (latitude / cell_size).astype(int) * cell_size
    cell_longitude = (longitude / cell_size).astype(int) * cell_size
    
    return cell_latitude, cell_longitude

# Apply the function to assign coordinates to cells
df['cell_lat'], df['cell_lon'] = assign_to_cells(df['lat'], df['lon'])
df.to_csv('cleaned.csv')

In [3]:
import pandas as pd
df = pd.read_csv('cleaned.csv')
df['cell_lat'] = round(df['cell_lat'], 6)
df['cell_lon'] = round(df['cell_lon'], 6)
df.to_csv('cleaned2.csv', index=False)

In [4]:
df.isna().sum()

Unnamed: 0    0
lat           0
lon           0
cell_lat      0
cell_lon      0
dtype: int64

In [5]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df.to_csv('cleaned2.csv', index=False)

In [8]:
import pandas as pd
df = pd.read_csv('cleaned2.csv')
adf = pd.read_csv('elevation.csv')
merged_df = pd.merge(df, adf, on=['cell_lat', 'cell_lon'], how='left')
merged_df.to_csv('cleaned2.csv', index=False)

In [9]:
df = pd.read_csv('cleaned2.csv')
df.drop(['cell_lat', 'cell_lon'], axis=1, inplace=True)
df.to_csv('cleaned2.csv', index=False)

In [10]:
import pandas as pd
df = pd.read_csv('grouped.csv')
adf = pd.read_csv('cleaned2.csv')
merged_df = pd.merge(df, adf, on=['lat', 'lon'], how='left')
merged_df.to_csv('grouped2.csv', index=False)